Example script for parsing. 

In [7]:
from parsing import * 

#Partial dataset
data_raw_fname = 'be223a_dataset.csv'
encoding=None
dtformat='%m/%d/%Y %H:%M'
patients, appointments, weather=parsing(data_raw_fname, encoding, dtformat, 'Exam ID','Patient ID', 'Age', 'Gender')

display(patients.head())
display(appointments.head())
display(weather.head())

Reading be223a_dataset.csv
Processed in 3.736 seconds.


,Patient_ID,Age,Gender
0,0000219e4b37d2504fb6b8c28e24a2d4,85,F
1,0000a7ddc7607d1d8ee5c47dd437034e,31,F
2,0000e1b68421441f1bb8e697ecdad119,61,F
3,00028fae880618d73d2dcfa3e5bc670b,68,F
5,0002aac1c9addac1d4ee6902d6ec5404,64,F


,Exam_ID,Patient_ID,OrgCode,Modality,Anatomy,SubSpecialty,DepartmentCode,Datetime_Obj,Dayofyear,ICD_Group,Label,Timeofday,Weekday
0,8637381,0000219e4b37d2504fb6b8c28e24a2d4,SMWG,US,LWREXT,US,US,3246-11-27 13:00:00,331.0,musculoskeletal,0,PM,1.0
1,8596175,0000a7ddc7607d1d8ee5c47dd437034e,MP,CR,CHEST,CHEST,GEN,3246-11-01 14:10:00,305.0,circulatory,0,PM,3.0
2,8756880,0000e1b68421441f1bb8e697ecdad119,SMWG,MG,LTBREAST,BREAST,MAM,NaN,365.0,nan,0,OFF,NaN
3,8655662,00028fae880618d73d2dcfa3e5bc670b,SMWG,MG,BREAST,BREAST,MAM,3246-12-06 09:40:00,340.0,nan,0,AM,3.0
4,8655670,00028fae880618d73d2dcfa3e5bc670b,SMWG,MG,BREAST,BREAST,MAM,3246-12-06 09:45:00,340.0,nan,0,AM,3.0


,mintemp,maxtemp,avtemp,precip,Dayofyear
0,48.0,68.0,58.0,8.0,0
1,48.0,68.0,58.0,9.0,1
2,48.0,68.0,58.0,9.0,2
3,48.0,68.0,58.0,9.0,3
4,48.0,68.0,58.0,9.0,4


In [8]:
from parsing import * 

#Full dataset
data_raw_fname = 'be223a_dataset_full.csv'
encoding="ISO-8859-1"
dtformat='%Y-%m-%d %H:%M:%S'
patients, appointments, weather = parsing(data_raw_fname, encoding, dtformat, 
                                          exam_id = 'examId', pt_id = 'patient_deid', age = 'AgeAtExam', gender = 'Gender')

display(patients.head())
display(appointments.head())
display(weather.head())

Reading be223a_dataset_full.csv
Processed in 6.408 seconds.


,Patient_ID,Age,Gender
0,12eb51a27782ce3071e281eeec4a69f6,52,M
3,b0bda126f56c50aa01109c57b7951599,31,F
5,82bfa53cd88960053dd9ae2ac8444bfc,47,F
6,5a1c7bb17d6200b541e202c06406b7d3,46,F
7,4f29344aa44f3661827af0a8ab01568e,49,F


,Exam_ID,Patient_ID,OrgCode,Modality,Anatomy,SubSpecialty,DepartmentCode,Datetime_Obj,Dayofyear,ICD_Group,Label,Timeofday,Weekday
0,8062697,12eb51a27782ce3071e281eeec4a69f6,MP1,MR,BRAIN,NEURO,MRI,3246-01-27 09:20:00,27.0,nan,0,AM,5.0
1,8062696,12eb51a27782ce3071e281eeec4a69f6,MP1,CT,BRAIN,NEURO,CT,3246-01-27 09:00:00,27.0,nan,0,AM,5.0
2,8062798,12eb51a27782ce3071e281eeec4a69f6,AWW,US,HEART,ECHO2020,CI2020,3246-01-04 17:00:00,4.0,nan,0,OFF,3.0
3,8062299,b0bda126f56c50aa01109c57b7951599,SMWG,MG,LTBREAST,BREAST,MAM,3246-01-04 13:20:00,4.0,nan,0,PM,3.0
4,8062300,b0bda126f56c50aa01109c57b7951599,SMWG,MG,RTBREAST,BREAST,MAM,3246-01-04 13:20:00,4.0,nan,0,PM,3.0


,mintemp,maxtemp,avtemp,precip,Dayofyear
0,48.0,68.0,58.0,8.0,0
1,48.0,68.0,58.0,9.0,1
2,48.0,68.0,58.0,9.0,2
3,48.0,68.0,58.0,9.0,3
4,48.0,68.0,58.0,9.0,4
